<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Work/blob/main/07_10_22_Fasttext_XGB_200full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U xgboost wandb fasttext pyfasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 192.9 MB 63 kB/s 
     |████████████████████████████████| 1.8 MB 43.1 MB/s 
     |████████████████████████████████| 68 kB 4.0 MB/s 
     |████████████████████████████████| 244 kB 13.8 MB/s 
     |████████████████████████████████| 145 kB 63.8 MB/s 
     |████████████████████████████████| 181 kB 54.4 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
  Using cached pybind11-2.9.2-py2.py3-none-any.whl (213 kB)
  Using cached cysignals-1.11.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (871 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3138275 sha256=b1509fa51cfe1a6d2e306a2cefccc75ed7d2272a6cbb64049b4ab1a2f1746ba7
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
  Created wheel for pyfasttext: filename=pyfasttext-0.4.6-cp37-cp3

In [2]:
import pandas as pd
import numpy as np
import nltk
import gensim
from gensim.models import Word2Vec, FastText
import fasttext as ft
import string
nltk.download('stopwords')
import re
import spacy
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import xgboost as xgb
from pathlib import Path

import wandb
from wandb.xgboost import WandbCallback
from IPython.display import clear_output


metrics = [accuracy_score, f1_score, precision_score, recall_score]

def get_name(score_func):
    return 'eval/' + score_func.__name__.split("_")[0]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
base_dir = Path("/content/drive/MyDrive/ResearchFND")
assert base_dir.exists()

In [5]:
df = pd.read_csv(base_dir/"gossipcop_agg.csv", index_col=0)
df.head()

,text,tweets,retweets,label,url
title,,,,,
Kendall Kylie Jenner Jenner NOT Upset Upset Over Caitlyn Engagement Engagement Despite Report,NaN,[],"['995423424741888001', '995461685166202880', '...",fake,NaN
Kim Kardashian Dethroned Dethroned By Khloe As Hot Hot Sister,NaN,[],"['848843565027516416', '849030801970868224', '...",fake,NaN
Kim Kardashian Did NOT Hot Staffer Hot Staffer To Save Marriage Save Marriage Despite Claim,NaN,[],"['940685393112064001', '977921622672920576', '...",fake,NaN
The Voice The Voice Team NOT Surprised Surprised Adam Levine Behati Prinsloo Pregnant Again Before Blake Shelton Gwen Stefani,NaN,[],[],fake,NaN
Drake NOT Angelina Jolie s Toy Boy Toy Boy Despite Late And Wrong Report,NaN,"[{'id': 948630026496323585, 'text': 'Drake NOT...","['948022124626808832', '948630026496323585', '...",fake,NaN


In [9]:
# NLP Preprocessing
# commenting since embeddings are created already: from gensim.utils import simple_preprocess

# NLP Preprocess - gensim.utils.simple_preprocess(doc, deacc=False, min_len=2, max_len=15)[source]
# Convert a document into a list of tokens.
# This lowercases, tokenizes, de-accents (optional). – the output are final tokens = unicode strings, that won’t be processed any further.

# commenting since embeddings are created already: df["title"] = df.title.map(lambda x: ' '.join(simple_preprocess(x)))

In [ ]:
# %%capture
# commenting since embeddings are created already: !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
# commenting since embeddings are created already: !gunzip cc.en.300.bin.gz

--2022-06-25 10:46:13--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: ‘cc.en.300.bin.gz’

cc.en.300.bin.gz    100%[===================>]   4.19G  63.8MB/s    in 86s     

2022-06-25 10:47:39 (50.1 MB/s) - ‘cc.en.300.bin.gz’ saved [4503593528/4503593528]



trying to see similarity of vectors created.

In [6]:
# Loading model for English
# commenting since embeddings are created already : modelog = ft.load_model('cc.en.300.bin')

In [7]:
ft_title_emb_file = base_dir/"gossipcop_ft_title_embeddings.npy"

if ft_title_emb_file.exists():
    title_embeddings = np.load(ft_title_emb_file)
else:
    title_embeddings = []
    for line in df['title']:
        emb = modelog.get_sentence_vector(line)
        title_embeddings.append(emb)
    title_embeddings = np.stack(title_embeddings)
    np.save(ft_title_emb_file, title_embeddings)

In [8]:
ft_text_emb_file = base_dir/"gossipcop_ft_text_embeddings.npy"
text_embeddings = np.load(ft_text_emb_file)



In [9]:
# commenting since embeddings are created alreadyif ft_text_emb_file.exists():
    #text_embeddings = np.load(ft_text_emb_file)
#else:
   # df.text.fillna("", inplace=True)
  # texts = df.title + " " + df.text.map(lambda x: " ".join(simple_preprocess(x)))
   # text_embeddings = []
   # for line in texts:
       # emb = modelog.get_sentence_vector(line)
     #   text_embeddings.append(emb)
   # text_embeddings = np.stack(text_embeddings)
  #  np.save(ft_text_emb_file, text_embeddings)

In [10]:
#import gc

#del modelog; gc.collect()

### Title embeddings

In [11]:
X = title_embeddings
y = (df.label=="real").to_numpy().astype(int)

X.shape, y.shape

((19968, 300), (19968,))

In [12]:
skf = StratifiedKFold(shuffle=True, random_state=124)

In [13]:
def train(train_idx, test_idx, params):

    # training
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    watchlist = [(dtrain,'train'), (dtest,'eval')]
    clf = xgb.train(params, dtrain, num_boost_round=201, early_stopping_rounds=None, evals=watchlist, callbacks=[WandbCallback()])
    #evaluation
    probs = clf.predict(dtest)
    y_pred = (probs > 0.5).astype(int)
    eval_results = {get_name(f):f(y_pred=y_pred, y_true=y_test) for f in metrics}
    wandb.log(eval_results)
    wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None,
                            y_true=y_test, preds=y_pred,
                            class_names=["Fake", "Real"])})

In [14]:
params = {
    "objective":'binary:logistic',
    "seed":124
}

In [15]:
GROUP = "gossipcop_fasttext-title-xgb"

for fold_id, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    clear_output()
    with wandb.init(entity="saloniteam", project="nofolds", group=GROUP, name=f"{GROUP}-fold-{fold_id}") as run:
        train(train_idx, test_idx, params)

[0]	train-logloss:0.58276	eval-logloss:0.59422
[1]	train-logloss:0.51648	eval-logloss:0.53977
[2]	train-logloss:0.47231	eval-logloss:0.50871
[3]	train-logloss:0.43940	eval-logloss:0.48640
[4]	train-logloss:0.41638	eval-logloss:0.47405
[5]	train-logloss:0.39531	eval-logloss:0.46268
[6]	train-logloss:0.37909	eval-logloss:0.45462
[7]	train-logloss:0.36493	eval-logloss:0.44673
[8]	train-logloss:0.35171	eval-logloss:0.44268
[9]	train-logloss:0.33915	eval-logloss:0.43893
[10]	train-logloss:0.32976	eval-logloss:0.43599
[11]	train-logloss:0.31957	eval-logloss:0.43328
[12]	train-logloss:0.30926	eval-logloss:0.43239
[13]	train-logloss:0.30090	eval-logloss:0.43119
[14]	train-logloss:0.29307	eval-logloss:0.43046
[15]	train-logloss:0.28538	eval-logloss:0.42974
[16]	train-logloss:0.27642	eval-logloss:0.42704
[17]	train-logloss:0.26978	eval-logloss:0.42586
[18]	train-logloss:0.26255	eval-logloss:0.42643
[19]	train-logloss:0.25504	eval-logloss:0.42704
[20]	train-logloss:0.24653	eval-logloss:0.42599
[2

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval-logloss,█▃▁▁▁▁▁▁▁▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇
eval/accuracy,▁
eval/f1,▁
eval/precision,▁
eval/recall,▁
train-logloss,█▆▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,200
eval/accuracy,0.82945
eval/f1,0.89658
eval/precision,0.84367


### Text embeddings

In [16]:
X = text_embeddings
y = (df.label=="real").to_numpy().astype(int)

X.shape, y.shape

((19968, 300), (19968,))

In [17]:
skf = StratifiedKFold(shuffle=True, random_state=124)

In [18]:
def train(train_idx, test_idx, params):

    # training
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    watchlist = [(dtrain,'train'), (dtest,'eval')]
    clf = xgb.train(params, dtrain, num_boost_round=200, early_stopping_rounds=None, evals=watchlist, callbacks=[WandbCallback()])
    #evaluation
    probs = clf.predict(dtest)
    y_pred = (probs > 0.5).astype(int)
    eval_results = {get_name(f):f(y_pred=y_pred, y_true=y_test) for f in metrics}
    wandb.log(eval_results)
    wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None,
                            y_true=y_test, preds=y_pred,
                            class_names=["Fake", "Real"])})

In [19]:
params = {
    "objective":'binary:logistic',
    "seed":124
}

In [20]:
GROUP = "gossipcop_fastext-fulltext-xgb"
for fold_id, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    clear_output()
    with wandb.init(entity="saloniteam", project="nofolds", group=GROUP, name=f"{GROUP}-fold-{fold_id}") as run:
        train(train_idx, test_idx, params)

[0]	train-logloss:0.57360	eval-logloss:0.58210
[1]	train-logloss:0.50155	eval-logloss:0.52081
[2]	train-logloss:0.45552	eval-logloss:0.48167
[3]	train-logloss:0.42276	eval-logloss:0.45677
[4]	train-logloss:0.39702	eval-logloss:0.43884
[5]	train-logloss:0.37652	eval-logloss:0.42676
[6]	train-logloss:0.35917	eval-logloss:0.41959
[7]	train-logloss:0.34445	eval-logloss:0.41159
[8]	train-logloss:0.33272	eval-logloss:0.40691
[9]	train-logloss:0.32103	eval-logloss:0.40331
[10]	train-logloss:0.31037	eval-logloss:0.39944
[11]	train-logloss:0.30164	eval-logloss:0.39746
[12]	train-logloss:0.29359	eval-logloss:0.39667
[13]	train-logloss:0.28510	eval-logloss:0.39430
[14]	train-logloss:0.27789	eval-logloss:0.39381
[15]	train-logloss:0.27105	eval-logloss:0.39331
[16]	train-logloss:0.26155	eval-logloss:0.39148
[17]	train-logloss:0.25558	eval-logloss:0.39039
[18]	train-logloss:0.24936	eval-logloss:0.38947
[19]	train-logloss:0.24526	eval-logloss:0.38703
[20]	train-logloss:0.23932	eval-logloss:0.38660
[2

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval-logloss,█▃▂▂▁▁▁▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆
eval/accuracy,▁
eval/f1,▁
eval/precision,▁
eval/recall,▁
train-logloss,█▆▅▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,199
eval/accuracy,0.84773
eval/f1,0.90652
eval/precision,0.86249
